In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #支持多条语句一起输出

import pandas as pd
import numpy as np
import quantaxis_ext
import matplotlib.pyplot as plt
import datetime
import settings
import calculator

#使用的指数代码
zs_symbol='000300'

In [3]:
#指数日线数据
zs_daily=quantaxis_ext.fetch_index_day_adv(zs_symbol,settings.stock_daily_start,settings.stock_daily_end)

In [6]:
#指数日收益
zs_daily_return=calculator.calc_daily_return(zs_daily['close'])

In [30]:
stock_list=quantaxis_ext.fetch_stock_list()
stock_list.head(1)

,code,decimal_point,name,pre_close,sec,sse,volunit
code,,,,,,,
000001,000001,2,平安银行,456.0,stock_cn,sz,100


In [31]:
stock_list=stock_list[stock_list['sse']=='sh'][['code','name']].set_index('code')

In [32]:
stock_list

,name
code,
600000,浦发银行
600004,白云机场
600006,东风汽车
600007,中国国贸
600008,首创股份
600009,上海机场
600010,包钢股份
600011,华能国际
600012,皖通高速


In [18]:
%%time

daily_return_dict={}
for code in stock_list.index:
    stock_daily=quantaxis_ext.fetch_stock_day_adv(code,settings.stock_daily_start,settings.stock_daily_end)
    if stock_daily.empty:
        continue
    stock_daily_return=calculator.calc_daily_return(stock_daily['close'])
    daily_return_dict[code]=daily_return=pd.DataFrame({zs_symbol:zs_daily_return,code:stock_daily_return})

QA Error QA_fetch_stock_day_adv parameter code=601298 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601615 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601860 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601975 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=603121 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=603332 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=603700 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=603739 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None


In [20]:
daily_return_dict['600000'].head()

,000300,600000
date,,
1999-11-11,NaN,-0.001441
1999-11-12,NaN,0.012270
1999-11-15,NaN,-0.010695
1999-11-16,NaN,-0.043243
1999-11-17,NaN,0.023729


In [33]:
daily_return_dict['603999'].head()

,000300,603999
date,,
2005-01-05,0.009941,NaN
2005-01-06,-0.009460,NaN
2005-01-07,0.000793,NaN
2005-01-10,0.010082,NaN
2005-01-11,0.003280,NaN
